<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 2*
# Convolutional Neural Networks (CNNs)

# Assignment

- <a href="#p1">Part 1:</a> Pre-Trained Model
- <a href="#p2">Part 2:</a> Custom CNN Model
- <a href="#p3">Part 3:</a> CNN with Data Augmentation


You will apply three different CNN models to a binary image classification model using Keras. Classify images of Mountains (`./data/train/mountain/*`) and images of forests (`./data/train/forest/*`). Treat mountains as the positive class (1) and the forest images as the negative (zero). 

|Mountain (+)|Forest (-)|
|---|---|
|![](https://github.com/mahoryu/DS-Unit-4-Sprint-3-Deep-Learning/blob/master/module2-convolutional-neural-networks/data/train/mountain/art1131.jpg?raw=1)|![](https://github.com/mahoryu/DS-Unit-4-Sprint-3-Deep-Learning/blob/master/module2-convolutional-neural-networks/data/validation/forest/cdmc317.jpg?raw=1)|

The problem is relatively difficult given that the sample is tiny: there are about 350 observations per class. This sample size might be something that you can expect with prototyping an image classification problem/solution at work. Get accustomed to evaluating several different possible models.

# Pre - Trained Model
<a id="p1"></a>

Load a pretrained network from Keras, [ResNet50](https://tfhub.dev/google/imagenet/resnet_v1_50/classification/1) - a 50 layer deep network trained to recognize [1000 objects](https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt). Starting usage:

```python
import numpy as np

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model # This is the functional API

resnet = ResNet50(weights='imagenet', include_top=False)

```

The `include_top` parameter in `ResNet50` will remove the full connected layers from the ResNet model. The next step is to turn off the training of the ResNet layers. We want to use the learned parameters without updating them in future training passes. 

```python
for layer in resnet.layers:
    layer.trainable = False
```

Using the Keras functional API, we will need to additional additional full connected layers to our model. We we removed the top layers, we removed all preivous fully connected layers. In other words, we kept only the feature processing portions of our network. You can expert with additional layers beyond what's listed here. The `GlobalAveragePooling2D` layer functions as a really fancy flatten function by taking the average of each of the last convolutional layer outputs (which is two dimensional still). 

```python
x = resnet.output
x = GlobalAveragePooling2D()(x) # This layer is a really fancy flatten
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(resnet.input, predictions)
```

Your assignment is to apply the transfer learning above to classify images of Mountains (`./data/train/mountain/*`) and images of forests (`./data/train/forest/*`). Treat mountains as the positive class (1) and the forest images as the negative (zero). 

Steps to complete assignment: 
1. Load in Image Data into numpy arrays (`X`) 
2. Create a `y` for the labels
3. Train your model with pre-trained layers from resnet
4. Report your model's accuracy

## Load in Data

This surprisingly more difficult than it seems, because you are working with directories of images instead of a single file. This boiler plate will help you download a zipped version of the directory of images. The directory is organized into "train" and "validation" which you can use inside an `ImageGenerator` class to stream batches of images thru your model.  


### Download & Summarize the Data

This step is completed for you. Just run the cells and review the results. 

In [1]:
import tensorflow as tf
import os

_URL = 'https://github.com/LambdaSchool/DS-Unit-4-Sprint-3-Deep-Learning/blob/master/module2-convolutional-neural-networks/data.zip?raw=true'

path_to_zip = tf.keras.utils.get_file('./data.zip', origin=_URL, extract=True)
PATH = os.path.join(os.path.dirname(path_to_zip), 'data')

42172416/42170838 [==============================] - 0s 0us/step


In [0]:
train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

In [0]:
train_mountain_dir = os.path.join(train_dir, 'mountain')  # directory with our training mountain pictures
train_forest_dir = os.path.join(train_dir, 'forest')  # directory with our training forest pictures
validation_mountain_dir = os.path.join(validation_dir, 'mountain')  # directory with our validation mountain pictures
validation_forest_dir = os.path.join(validation_dir, 'forest')  # directory with our validation forest pictures

In [0]:
num_mountain_tr = len(os.listdir(train_mountain_dir))
num_forest_tr = len(os.listdir(train_forest_dir))

num_mountain_val = len(os.listdir(validation_mountain_dir))
num_forest_val = len(os.listdir(validation_forest_dir))

total_train = num_mountain_tr + num_forest_tr
total_val = num_mountain_val + num_forest_val

In [5]:
print('total training mountain images:', num_mountain_tr)
print('total training forest images:', num_forest_tr)

print('total validation mountain images:', num_mountain_val)
print('total validation forest images:', num_forest_val)
print("--")
print("Total training images:", total_train)
print("Total validation images:", total_val)

total training mountain images: 254
total training forest images: 270
total validation mountain images: 125
total validation forest images: 62
--
Total training images: 524
Total validation images: 187


### Keras `ImageGenerator` to Process the Data

This step is completed for you, but please review the code. The `ImageGenerator` class reads in batches of data from a directory and pass them to the model one batch at a time. Just like large text files, this method is advantageous, because it stifles the need to load a bunch of images into memory. 

Check out the documentation for this class method: [Keras `ImageGenerator` Class](https://keras.io/preprocessing/image/#imagedatagenerator-class). You'll expand it's use in the third assignment objective.

In [0]:
batch_size = 16
epochs = 50
IMG_HEIGHT = 224
IMG_WIDTH = 224

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our training data
validation_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our validation data

In [8]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='binary')

Found 533 images belonging to 2 classes.


In [9]:
val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=validation_dir,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='binary')

Found 195 images belonging to 2 classes.


## Instatiate Model

In [10]:
import numpy as np
 
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
 
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model # This is the functional API
 
resnet = ResNet50(weights='imagenet', include_top=False)

94773248/94765736 [==============================] - 3s 0us/step


In [0]:
for layer in resnet.layers:
    layer.trainable = False

In [0]:
x = resnet.output
x = GlobalAveragePooling2D()(x) # This layer is a really fancy flatten
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(resnet.input, predictions)

In [13]:
model.compile(optimizer='adam',
              loss= 'categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, None, None, 6 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

## Fit Model

In [14]:
history = model.fit(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size
)

Epoch 1/50
32/32 [==============================] - 4s 119ms/step - loss: 5.7344e-08 - accuracy: 0.4810 - val_loss: 7.5860e-08 - val_accuracy: 0.6364
Epoch 2/50
32/32 [==============================] - 3s 79ms/step - loss: 5.8058e-08 - accuracy: 0.4870 - val_loss: 7.9247e-08 - val_accuracy: 0.6648
Epoch 3/50
32/32 [==============================] - 3s 80ms/step - loss: 5.7582e-08 - accuracy: 0.4830 - val_loss: 7.8570e-08 - val_accuracy: 0.6591
Epoch 4/50
32/32 [==============================] - 3s 80ms/step - loss: 5.8296e-08 - accuracy: 0.4890 - val_loss: 7.7892e-08 - val_accuracy: 0.6534
Epoch 5/50
32/32 [==============================] - 3s 80ms/step - loss: 5.7509e-08 - accuracy: 0.4824 - val_loss: 7.7892e-08 - val_accuracy: 0.6534
Epoch 6/50
32/32 [==============================] - 3s 80ms/step - loss: 5.7582e-08 - accuracy: 0.4830 - val_loss: 7.8570e-08 - val_accuracy: 0.6591
Epoch 7/50
32/32 [==============================] - 3s 80ms/step - loss: 5.5917e-08 - accuracy: 0.4691 - 

# Custom CNN Model

In this step, write and train your own convolutional neural network using Keras. You can use any architecture that suits you as long as it has at least one convolutional and one pooling layer at the beginning of the network - you can add more if you want. 

In [0]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten

In [50]:
train_data_gen[0][0][0].shape

(224, 224, 3)

In [31]:
train_data_gen[0][1]

array([0., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 0., 1., 1., 0., 1.],
      dtype=float32)

In [60]:
# Compile Model
# Setup Architecture
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(224,224,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile Model
model.compile(optimizer='nadam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Print summary
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_30 (Conv2D)           (None, 222, 222, 32)      896       
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 111, 111, 32)      0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 394272)            0         
_________________________________________________________________
dense_24 (Dense)             (None, 64)                25233472  
_________________________________________________________________
dense_25 (Dense)             (None, 1)                 65        
Total params: 25,234,433
Trainable params: 25,234,433
Non-trainable params: 0
_________________________________________________________________


In [61]:
# Fit Model
model.fit(train_data_gen, epochs=10, validation_data=(val_data_gen))

Epoch 1/10
34/34 [==============================] - 2s 67ms/step - loss: 2.7532 - accuracy: 0.7899 - val_loss: 0.2556 - val_accuracy: 0.9026
Epoch 2/10
34/34 [==============================] - 2s 66ms/step - loss: 0.1572 - accuracy: 0.9456 - val_loss: 0.2951 - val_accuracy: 0.8974
Epoch 3/10
34/34 [==============================] - 2s 65ms/step - loss: 0.1042 - accuracy: 0.9644 - val_loss: 0.6651 - val_accuracy: 0.8564
Epoch 4/10
34/34 [==============================] - 2s 64ms/step - loss: 0.0753 - accuracy: 0.9775 - val_loss: 0.2864 - val_accuracy: 0.9231
Epoch 5/10
34/34 [==============================] - 2s 67ms/step - loss: 0.0185 - accuracy: 0.9981 - val_loss: 0.1715 - val_accuracy: 0.9282
Epoch 6/10
34/34 [==============================] - 2s 64ms/step - loss: 0.0082 - accuracy: 1.0000 - val_loss: 0.1903 - val_accuracy: 0.9385
Epoch 7/10
34/34 [==============================] - 2s 65ms/step - loss: 0.0031 - accuracy: 1.0000 - val_loss: 0.2012 - val_accuracy: 0.9333
Epoch 8/10
34

# Custom CNN Model with Image Manipulations

To simulate an increase in a sample of image, you can apply image manipulation techniques: cropping, rotation, stretching, etc. Luckily Keras has some handy functions for us to apply these techniques to our mountain and forest example. Simply, you should be able to modify our image generator for the problem. Check out these resources to help you get started: 

1. [Keras `ImageGenerator` Class](https://keras.io/preprocessing/image/#imagedatagenerator-class)
2. [Building a powerful image classifier with very little data](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html)
 

In [0]:
batch_size = 16
epochs = 50
IMG_HEIGHT = 224
IMG_WIDTH = 224

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_image_generator = ImageDataGenerator(rescale=1./255,
                                           shear_range=0.2,
                                           zoom_range=0.2,
                                           horizontal_flip=True,
                                           width_shift_range=0.2,
                                           height_shift_range=0.2) # Generator for our training data


validation_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our validation data

In [101]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='binary')

Found 533 images belonging to 2 classes.


In [102]:
val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=validation_dir,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='binary')

Found 195 images belonging to 2 classes.


In [110]:
from tensorflow.keras.callbacks import EarlyStopping

# Compile Model
# Setup Architecture
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(224,224,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile Model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Print summary
model.summary()

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_55 (Conv2D)           (None, 222, 222, 32)      896       
_________________________________________________________________
max_pooling2d_40 (MaxPooling (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_56 (Conv2D)           (None, 109, 109, 64)      18496     
_________________________________________________________________
flatten_23 (Flatten)         (None, 760384)            0         
_________________________________________________________________
dense_50 (Dense)             (None, 64)                48664640  
_________________________________________________________________
dense_51 (Dense)             (None, 1)                 65        
Total params: 48,684,097
Trainable params: 48,684,097
Non-trainable params: 0
_________________________________________

In [98]:
# # Fit Model
# model.fit(train_data_gen, epochs=15, validation_data=(val_data_gen),callbacks=[EarlyStopping(min_delta=.02, monitor='val_accuracy', patience=3)])

Epoch 1/15
34/34 [==============================] - 9s 268ms/step - loss: 2.5635 - accuracy: 0.7824 - val_loss: 0.3082 - val_accuracy: 0.8462
Epoch 2/15
34/34 [==============================] - 9s 263ms/step - loss: 0.2960 - accuracy: 0.8931 - val_loss: 0.3143 - val_accuracy: 0.8667
Epoch 3/15
34/34 [==============================] - 9s 266ms/step - loss: 0.2961 - accuracy: 0.8949 - val_loss: 0.2236 - val_accuracy: 0.9179
Epoch 4/15
34/34 [==============================] - 9s 265ms/step - loss: 0.2259 - accuracy: 0.9043 - val_loss: 0.1600 - val_accuracy: 0.9179
Epoch 5/15
34/34 [==============================] - 9s 262ms/step - loss: 0.1993 - accuracy: 0.9137 - val_loss: 0.3378 - val_accuracy: 0.8205
Epoch 6/15
34/34 [==============================] - 9s 261ms/step - loss: 0.2522 - accuracy: 0.8856 - val_loss: 0.2509 - val_accuracy: 0.9179


In [111]:
model.fit(train_data_gen,
          steps_per_epoch=total_train // batch_size,
          epochs=epochs,
          validation_data=val_data_gen,
          validation_steps=total_val // batch_size,
          callbacks=[EarlyStopping(min_delta=.02, monitor='val_loss', patience=5)])

Epoch 1/50
32/32 [==============================] - 7s 216ms/step - loss: 1.6545 - accuracy: 0.7844 - val_loss: 0.6045 - val_accuracy: 0.7273
Epoch 2/50
32/32 [==============================] - 7s 211ms/step - loss: 0.3260 - accuracy: 0.8842 - val_loss: 0.6313 - val_accuracy: 0.7045
Epoch 3/50
32/32 [==============================] - 7s 212ms/step - loss: 0.2742 - accuracy: 0.8942 - val_loss: 0.6752 - val_accuracy: 0.7330
Epoch 4/50
32/32 [==============================] - 7s 211ms/step - loss: 0.2080 - accuracy: 0.9182 - val_loss: 0.2383 - val_accuracy: 0.9091
Epoch 5/50
32/32 [==============================] - 7s 213ms/step - loss: 0.2366 - accuracy: 0.9022 - val_loss: 0.6542 - val_accuracy: 0.7727
Epoch 6/50
32/32 [==============================] - 7s 211ms/step - loss: 0.2492 - accuracy: 0.9042 - val_loss: 0.8278 - val_accuracy: 0.7727
Epoch 7/50
32/32 [==============================] - 7s 213ms/step - loss: 0.1805 - accuracy: 0.9441 - val_loss: 0.7732 - val_accuracy: 0.7330
Epoch 

# Resources and Stretch Goals

Stretch goals
- Enhance your code to use classes/functions and accept terms to search and classes to look for in recognizing the downloaded images (e.g. download images of parties, recognize all that contain balloons)
- Check out [other available pretrained networks](https://tfhub.dev), try some and compare
- Image recognition/classification is somewhat solved, but *relationships* between entities and describing an image is not - check out some of the extended resources (e.g. [Visual Genome](https://visualgenome.org/)) on the topic
- Transfer learning - using images you source yourself, [retrain a classifier](https://www.tensorflow.org/hub/tutorials/image_retraining) with a new category
- (Not CNN related) Use [piexif](https://pypi.org/project/piexif/) to check out the metadata of images passed in to your system - see if they're from a national park! (Note - many images lack GPS metadata, so this won't work in most cases, but still cool)

Resources
- [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385) - influential paper (introduced ResNet)
- [YOLO: Real-Time Object Detection](https://pjreddie.com/darknet/yolo/) - an influential convolution based object detection system, focused on inference speed (for applications to e.g. self driving vehicles)
- [R-CNN, Fast R-CNN, Faster R-CNN, YOLO](https://towardsdatascience.com/r-cnn-fast-r-cnn-faster-r-cnn-yolo-object-detection-algorithms-36d53571365e) - comparison of object detection systems
- [Common Objects in Context](http://cocodataset.org/) - a large-scale object detection, segmentation, and captioning dataset
- [Visual Genome](https://visualgenome.org/) - a dataset, a knowledge base, an ongoing effort to connect structured image concepts to language